In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import GridSearchCV

class EstimatorSelectionHelper:
    
    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
    
    def fit(self, X, y, **grid_kwargs):
        for key in self.keys:
            print('Running GridSearchCV for %s.' % key)
            model = self.models[key]
            params = self.params[key]
            grid_search = GridSearchCV(model, params, **grid_kwargs)
            grid_search.fit(X, y)
            self.grid_searches[key] = grid_search
        print('Done.')
    
    def score_summary(self, sort_by='mean_test_score'):
        frames = []
        for name, grid_search in self.grid_searches.items():
            frame = pd.DataFrame(grid_search.cv_results_)
            frame = frame.filter(regex='^(?!.*param_).*$')
            frame['estimator'] = len(frame)*[name]
            frames.append(frame)
        df = pd.concat(frames)
        
        df = df.sort_values([sort_by], ascending=False)
        df = df.reset_index()
        df = df.drop(['rank_test_score', 'index'], 1)
        
        columns = df.columns.tolist()
        columns.remove('estimator')
        columns = ['estimator']+columns
        df = df[columns]
        return df

In [3]:
from sklearn import datasets

breast_cancer = datasets.load_breast_cancer()
X_cancer = breast_cancer.data
y_cancer = breast_cancer.target

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

models1 = { 
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier()
}

params1 = { 
    'ExtraTreesClassifier': { 'n_estimators': [16, 32] },
    'RandomForestClassifier': [
        { 'n_estimators': [16, 32] },
        {'criterion': ['gini', 'entropy'], 'n_estimators': [8, 16]}],
    'AdaBoostClassifier':  { 'n_estimators': [16, 32] },
    'GradientBoostingClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] }
}

In [4]:
helper1 = EstimatorSelectionHelper(models1, params1)
helper1.fit(X_cancer, y_cancer, scoring='f1', n_jobs=2)
helper1.score_summary()

Running GridSearchCV for ExtraTreesClassifier.


/Users/jung-kyoyoon/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Running GridSearchCV for RandomForestClassifier.


/Users/jung-kyoyoon/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Running GridSearchCV for AdaBoostClassifier.


...

Running GridSearchCV for GradientBoostingClassifier.


/Users/jung-kyoyoon/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Done.


/Users/jung-kyoyoon/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/jung-kyoyoon/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/jung-kyoyoon/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/jun

,estimator,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,AdaBoostClassifier,0.055023,0.000837,0.003615,0.000535,{'n_estimators': 32},0.962343,0.991667,0.970711,0.974914,0.012343,1.000000,1.000000,1.000000,1.000000,0.000000
1,RandomForestClassifier,0.016426,0.000598,0.001781,0.000064,{'n_estimators': 16},0.957627,0.979253,0.983051,0.973293,0.011200,1.000000,1.000000,1.000000,1.000000,0.000000
2,ExtraTreesClassifier,0.018322,0.004839,0.003436,0.001136,{'n_estimators': 16},0.958333,0.979253,0.978723,0.972092,0.009744,1.000000,1.000000,1.000000,1.000000,0.000000
3,ExtraTreesClassifier,0.020994,0.000483,0.002880,0.000015,{'n_estimators': 32},0.966387,0.975207,0.969957,0.970518,0.003626,1.000000,1.000000,1.000000,1.000000,0.000000
4,GradientBoostingClassifier,0.021497,0.001267,0.000920,0.000179,"{'learning_rate': 0.8, 'n_estimators': 16}",0.955102,0.979253,0.970464,0.968269,0.009989,1.000000,1.000000,1.000000,1.000000,0.000000
5,AdaBoostClassifier,0.028793,0.000536,0.001999,0.000018,{'n_estimators': 16},0.963265,0.983333,0.952790,0.966487,0.012671,0.997904,0.997904,1.000000,0.998602,0.000988
6,RandomForestClassifier,0.011013,0.000550,0.001428,0.000198,"{'criterion': 'entropy', 'n_estimators': 8}",0.950413,0.979079,0.965812,0.965100,0.011724,0.995816,0.997904,1.000000,0.997906,0.001708
7,RandomForestClassifier,0.031717,0.001407,0.002581,0.000090,{'n_estimators': 32},0.945607,0.979253,0.970464,0.965099,0.014259,1.000000,1.000000,1.000000,1.000000,0.000000
8,GradientBoostingClassifier,0.018302,0.000172,0.000599,0.000023,"{'learning_rate': 1.0, 'n_estimators': 16}",0.954733,0.975207,0.962025,0.963992,0.008480,1.000000,1.000000,1.000000,1.000000,0.000000
9,RandomForestClassifier,0.016685,0.000475,0.001609,0.000026,"{'criterion': 'entropy', 'n_estimators': 16}",0.950413,0.979253,0.961039,0.963573,0.011919,1.000000,1.000000,1.000000,1.000000,0.000000
